# CNN Trainig Templete
## Based on Tensorlfow - CNN MNIST example
https://github.com/tensorflow/tensorflow/blob/r1.6/tensorflow/examples/tutorials/layers/cnn_mnist.py

## 필요한 library 호출

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib.layers.python.layers import layers as layers_lib
from tensorflow.python.ops import array_ops

import cv2
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random

import json

tf.logging.set_verbosity(tf.logging.INFO)


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data_txt = open('./data_split/data_dir_0801.txt', 'r')
data_json = data_txt.read()
tr_data_dir = json.loads(data_json)

In [3]:
# Data info.

width = 640
height = 480
channel = 3


# resize
r_w = 224
r_h = 224

total_pix = r_w * r_h * channel

In [4]:
tr_data_dir['train'][0]

'/dataset_full/Torn/Torn (780).png'

In [5]:
tr_data = np.zeros((len(tr_data_dir['train']), total_pix), dtype=np.float32)
tr_label = np.zeros((len(tr_data_dir['train']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['train'])):
    img = cv2.imread(tr_data_dir['train'][i])
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    tr_data[i,:] = img2.flatten()
    
    if 'Untorn' in tr_data_dir['train'][i]:
        tr_label[i] = 0
    elif 'normal' in tr_data_dir['train'][i]:
        tr_label[i] = 0
        
    else:
        tr_label[i] = 1

In [6]:
ii = 766
print(tr_data_dir['train'][ii], tr_label[ii])
print(len(tr_data_dir['train']))

/dataset_full/Torn/Torn (1731).png [1]
3648


In [7]:
def vgg16(features, labels, mode):
    
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, height, width, channels]
    # Our Fishing net image size is 640x480 and 3-channel (RGB)
    input_layer = tf.reshape(features["x"], [-1, 224, 224, 3])
    dropout_keep_prob = 0.5
    num_classes = 2
    is_training = True
    
    
    net = layers_lib.repeat(
        input_layer, 2, layers.conv2d, 64, [3, 3], scope='conv1')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool1')
    net = layers_lib.repeat(net, 2, layers.conv2d, 128, [3, 3], scope='conv2')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool2')
    net = layers_lib.repeat(net, 3, layers.conv2d, 256, [3, 3], scope='conv3')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool3')
    net = layers_lib.repeat(net, 3, layers.conv2d, 512, [3, 3], scope='conv4')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool4')
    net = layers_lib.repeat(net, 3, layers.conv2d, 512, [3, 3], scope='conv5')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool5')
    # Use conv2d instead of fully_connected layers.
    net = layers.conv2d(net, 4096, [7, 7], padding='VALID', scope='fc6')
    net = layers_lib.dropout(
        net, dropout_keep_prob, is_training=is_training, scope='dropout6')
    net = layers.conv2d(net, 4096, [1, 1], scope='fc7')
    net = layers_lib.dropout(
        net, dropout_keep_prob, is_training=is_training, scope='dropout7')
    net = layers.conv2d(
        net,
        num_classes, [1, 1],
        activation_fn=None,
        normalizer_fn=None,
        scope='fc8')
    net = array_ops.squeeze(net, [1, 2], name='fc8/squeezed')
    
    logits = net
    

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    # sparse_softmax_cross_entropy cannot use one-hot encoding
    
    #loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [11]:
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.9
#session = tf.Session(config=config)
for i in range(9,10):
# Create the Estimator
    f_dir = "/models/ICROS_vgg/Train_%d/" % i

    if not tf.gfile.Exists(f_dir):
        tf.gfile.MakeDirs(f_dir)

    net_classifier = tf.estimator.Estimator(
        model_fn=vgg16, model_dir=f_dir)

    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=50)

    batch_s = 32
    epochs = 10
    tr_steps = int(tr_data.shape[0]/batch_s*epochs)
    in_steps = 1



    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": tr_data},
        y=tr_label,
        batch_size=batch_s,
        num_epochs=epochs,
        shuffle=False)
    net_classifier.train(
        input_fn=train_input_fn,
        steps=tr_steps,
        hooks=[logging_hook])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_num_worker_replicas': 1, '_tf_random_seed': None, '_service': None, '_evaluation_master': '', '_num_ps_replicas': 0, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f91d4818c18>, '_global_id_in_cluster': 0, '_is_chief': True, '_master': '', '_log_step_count_steps': 100, '_task_id': 0, '_save_checkpoints_steps': None, '_model_dir': '/models/ICROS_vgg/Train_9/', '_session_config': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /models/ICROS_vgg/Train_9/model.ckpt.
INFO:tensorflow:probabilities = [[0.

INFO:tensorflow:step = 401, loss = 0.23308945 (32.450 sec)
INFO:tensorflow:probabilities = [[0.98212737 0.01787266]
 [0.01995866 0.9800413 ]
 [0.02383528 0.9761647 ]
 [0.64472234 0.35527766]
 [0.3010409  0.6989591 ]
 [0.23426819 0.7657318 ]
 [0.32353735 0.67646265]
 [0.51077014 0.48922986]
 [0.99999976 0.00000021]
 [0.29564753 0.7043525 ]
 [0.29851264 0.70148736]
 [0.9997533  0.00024671]
 [0.3073004  0.6926997 ]
 [0.996104   0.00389598]
 [0.02021337 0.97978663]
 [0.99775416 0.00224591]
 [0.25986463 0.7401354 ]
 [0.02673067 0.9732693 ]
 [0.9999999  0.00000015]
 [0.20959121 0.79040873]
 [0.2633082  0.73669183]
 [0.14034416 0.85965586]
 [0.02571245 0.97428757]
 [0.99998784 0.00001213]
 [0.99766195 0.00233805]
 [0.2627308  0.7372692 ]
 [0.2076386  0.7923614 ]
 [0.99999857 0.00000139]
 [0.23324646 0.76675355]
 [0.99997115 0.00002885]
 [0.9998889  0.0001111 ]
 [0.99613416 0.00386587]] (16.186 sec)
INFO:tensorflow:global_step/sec: 3.08661
INFO:tensorflow:probabilities = [[0.9589941  0.0410059

INFO:tensorflow:step = 901, loss = 0.066867694 (32.301 sec)
INFO:tensorflow:probabilities = [[0.9999999  0.0000001 ]
 [0.         1.        ]
 [1.         0.        ]
 [1.         0.00000005]
 [0.00098061 0.9990194 ]
 [0.02004853 0.97995144]
 [0.00016431 0.9998356 ]
 [0.02716024 0.97283983]
 [1.         0.        ]
 [0.72739446 0.27260557]
 [0.01218462 0.9878154 ]
 [0.9208343  0.07916571]
 [0.9999995  0.0000005 ]
 [0.22730099 0.77269906]
 [0.999992   0.00000798]
 [0.43991303 0.56008697]
 [0.89982    0.10017995]
 [0.00000123 0.9999988 ]
 [0.00008934 0.9999106 ]
 [0.49560878 0.5043912 ]
 [0.14505792 0.8549421 ]
 [1.         0.        ]
 [0.         1.        ]
 [1.         0.        ]
 [0.25775892 0.7422411 ]
 [0.00023514 0.99976486]
 [0.00047069 0.99952936]
 [0.9572094  0.04279063]
 [0.9883343  0.0116657 ]
 [0.276775   0.723225  ]
 [0.9564411  0.04355897]
 [0.00002312 0.9999769 ]] (16.137 sec)
INFO:tensorflow:global_step/sec: 3.08366
INFO:tensorflow:probabilities = [[0.         1.      

In [9]:
"""
ev_results = np.zeros((len(ev_label), 1))


# Evaluate the model and print results
for i in range(len(ev_label)):
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": ev_data[i:i+1]},
        y=ev_label[i:i+1],
        num_epochs=1,
        shuffle=False)
    eval_results = net_classifier.evaluate(input_fn=eval_input_fn)

    ev_results[i] = eval_results["accuracy"]
    
"""

'\nev_results = np.zeros((len(ev_label), 1))\n\n\n# Evaluate the model and print results\nfor i in range(len(ev_label)):\n    eval_input_fn = tf.estimator.inputs.numpy_input_fn(\n        x={"x": ev_data[i:i+1]},\n        y=ev_label[i:i+1],\n        num_epochs=1,\n        shuffle=False)\n    eval_results = net_classifier.evaluate(input_fn=eval_input_fn)\n\n    ev_results[i] = eval_results["accuracy"]\n    \n'

In [10]:
#print(np.mean(ev_results))